### Multiclass Classification**
If the plane was delayed, we will predict what type of delay it is (will be).

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from ma_functions import *

In [2]:
con = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")

print("Database opened successfully")


Database opened successfully


In [3]:
def sql_execute(sql,con):
    cur = con.cursor()
    cur.execute(sql)
    print("Table created successfully")
    con.commit()
    con.close()

def sql_read(sql,con):
    return pd.read_sql(sql,con)

In [4]:
query='''
SELECT fl_date,
dep_time,
op_unique_carrier,
origin_airport_id,
dest_airport_id,
taxi_out,
arr_delay,
taxi_in,
distance,
air_time,
crs_elapsed_time,
origin_city_name,
dest_city_name,
crs_dep_time,
crs_arr_time,
dep_delay,
carrier_delay,
weather_delay,
nas_delay,
security_delay,
late_aircraft_delay
FROM flights
LIMIT 100000
'''
X_delays=sql_read(query,con)
X_delays

,fl_date,dep_time,op_unique_carrier,origin_airport_id,dest_airport_id,taxi_out,arr_delay,taxi_in,distance,air_time,...,origin_city_name,dest_city_name,crs_dep_time,crs_arr_time,dep_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-07-24,1615.0,UA,14635,13930,13.0,-21.0,17.0,1120.0,158.0,...,"Fort Myers, FL","Chicago, IL",1625,1844,-10.0,NaN,NaN,NaN,NaN,NaN
1,2018-07-24,1044.0,UA,12264,15304,22.0,125.0,4.0,811.0,112.0,...,"Washington, DC","Tampa, FL",847,1057,117.0,0.0,0.0,8.0,0.0,117.0
2,2018-07-24,816.0,UA,11618,13930,45.0,131.0,12.0,719.0,126.0,...,"Newark, NJ","Chicago, IL",639,808,97.0,97.0,0.0,34.0,0.0,0.0
3,2018-07-24,2125.0,UA,13930,12264,47.0,29.0,4.0,588.0,82.0,...,"Chicago, IL","Washington, DC",2123,9,2.0,0.0,0.0,29.0,0.0,0.0
4,2018-07-24,1422.0,UA,14107,13930,13.0,-3.0,20.0,1440.0,181.0,...,"Phoenix, AZ","Chicago, IL",1425,1959,-3.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2018-07-28,1227.0,OO,11433,11995,15.0,-10.0,7.0,460.0,70.0,...,"Detroit, MI","Greensboro/High Point, NC",1230,1409,-3.0,NaN,NaN,NaN,NaN,NaN
99996,2018-07-28,829.0,OO,11433,14730,17.0,-1.0,4.0,306.0,51.0,...,"Detroit, MI","Louisville, KY",820,942,9.0,NaN,NaN,NaN,NaN,NaN
99997,2018-07-28,1008.0,OO,14730,11433,18.0,-1.0,2.0,306.0,60.0,...,"Louisville, KY","Detroit, MI",1007,1129,1.0,NaN,NaN,NaN,NaN,NaN
99998,2018-07-28,1552.0,OO,13277,10397,12.0,-20.0,8.0,147.0,32.0,...,"Montgomery, AL","Atlanta, GA",1600,1804,-8.0,NaN,NaN,NaN,NaN,NaN


In [5]:
X_delays=getStatistics(X_delays)
X_delays.columns

Index(['fl_date', 'origin_airport_id', 'dep_hr', 'op_unique_carrier',
       'dep_time', 'dest_airport_id', 'taxi_out', 'arr_delay', 'taxi_in',
       'distance', 'air_time', 'crs_elapsed_time', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dep_delay',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'airport_30D_avg_dep_delay',
       'airport_30D_avg_arr_delay', 'airport_30D_min_dep_delay',
       'airport_30D_max_dep_delay', 'airport_30D_min_arr_delay',
       'airport_30D_max_arr_delay', 'carrier_30D_avg_dep_delay',
       'carrier_30D_avg_arr_delay', 'carrier_30D_min_dep_delay',
       'carrier_30D_max_dep_delay', 'carrier_30D_min_arr_delay',
       'carrier_30D_max_arr_delay', 'arr_hr', 'airport_30D_avg_taxi_out',
       'airport_30D_avg_taxi_in'],
      dtype='object')

In [9]:
weather_df=pd.read_csv('weather_data_complete.csv')
def getWeather(date,city):
    weather=0
    try:
        str=weather_df[weather_df['Date']==date][city].values[0]
    except:
        city=city[-2:]
        for i in weather_df.columns:
            if city in i:
                city=i
                break
        str=weather_df[weather_df['Date']==date][city].values[0]
            
    
    if 'Clear' in str:
        weather=0
    else:
        if 'Rain' in str:
            weather=2
        elif 'Snow' in str:
            weather=3
        else:
            weather=1
    return weather

In [10]:
X_delays['weather_code_org']=X_delays['fl_date'].combine(X_delays['origin_city_name'], func=getWeather)
X_delays['weather_code_dest']=X_delays['fl_date'].combine(X_delays['dest_city_name'], func=getWeather)

In [11]:
X_delays['airport_30D_avg_dep_delay']=X_delays['airport_30D_avg_dep_delay'].fillna(0)
X_delays['airport_30D_avg_arr_delay']=X_delays['airport_30D_avg_arr_delay'].fillna(0)
X_delays['airport_30D_min_dep_delay']=X_delays['airport_30D_min_dep_delay'].fillna(0)
X_delays['airport_30D_max_dep_delay']=X_delays['airport_30D_max_dep_delay'].fillna(0)
X_delays['airport_30D_min_arr_delay']=X_delays['airport_30D_min_arr_delay'].fillna(0)
X_delays['airport_30D_max_arr_delay']=X_delays['airport_30D_max_arr_delay'].fillna(0)

X_delays['carrier_30D_avg_dep_delay']=X_delays['airport_30D_avg_dep_delay'].fillna(0)
X_delays['carrier_30D_avg_arr_delay']=X_delays['carrier_30D_avg_arr_delay'].fillna(0)
X_delays['carrier_30D_min_dep_delay']= X_delays['carrier_30D_min_dep_delay'].fillna(0)
X_delays['carrier_30D_max_dep_delay']=X_delays['carrier_30D_max_dep_delay'].fillna(0)
X_delays['airport_30D_max_dep_delay']=X_delays['airport_30D_max_dep_delay'].fillna(0)
X_delays['carrier_30D_min_arr_delay']=X_delays['carrier_30D_min_arr_delay'].fillna(0)
X_delays['carrier_30D_max_arr_delay']=X_delays['carrier_30D_max_arr_delay'].fillna(0)

X_delays['airport_30D_avg_taxi_out']=X_delays['airport_30D_avg_taxi_out'].fillna(0)
X_delays['airport_30D_avg_taxi_in']=X_delays['airport_30D_avg_taxi_in'].fillna(0)

#X_delays.isnull().any()
X_delays=X_delays[~(X_delays['air_time'].isnull())]
X_delays=X_delays.reset_index(drop=True)

In [11]:
X_delays.columns

Index(['fl_date', 'origin_airport_id', 'dep_hr', 'op_unique_carrier',
       'dep_time', 'dest_airport_id', 'taxi_out', 'arr_delay', 'taxi_in',
       'distance', 'air_time', 'crs_elapsed_time', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dep_delay',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'airport_30D_avg_dep_delay',
       'airport_30D_avg_arr_delay', 'airport_30D_min_dep_delay',
       'airport_30D_max_dep_delay', 'airport_30D_min_arr_delay',
       'airport_30D_max_arr_delay', 'carrier_30D_avg_dep_delay',
       'carrier_30D_avg_arr_delay', 'carrier_30D_min_dep_delay',
       'carrier_30D_max_dep_delay', 'carrier_30D_min_arr_delay',
       'carrier_30D_max_arr_delay', 'arr_hr', 'airport_30D_avg_taxi_out',
       'airport_30D_avg_taxi_in', 'weather_code_org', 'weather_code_dest'],
      dtype='object')

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_delays['op_unique_carrier'])

LabelEncoder()

In [13]:
X_delays['carrier']=le.transform(X_delays['op_unique_carrier'])

In [14]:
X=X_delays[['fl_date', 'origin_airport_id', 'dep_hr', 'arr_hr','carrier','air_time', 'distance','dest_airport_id','distance','airport_30D_avg_dep_delay', 'airport_30D_avg_arr_delay',
       'airport_30D_min_dep_delay', 'airport_30D_max_dep_delay',
       'airport_30D_min_arr_delay', 'airport_30D_max_arr_delay',
       'carrier_30D_avg_dep_delay', 'carrier_30D_avg_arr_delay',
       'carrier_30D_min_dep_delay', 'carrier_30D_max_dep_delay',
       'carrier_30D_min_arr_delay', 'carrier_30D_max_arr_delay', 'airport_30D_avg_taxi_out',
       'airport_30D_avg_taxi_in','weather_code_org', 'weather_code_dest']]

In [15]:
X.isnull().sum()

fl_date                      0
origin_airport_id            0
dep_hr                       0
arr_hr                       0
carrier                      0
air_time                     0
distance                     0
dest_airport_id              0
distance                     0
airport_30D_avg_dep_delay    0
airport_30D_avg_arr_delay    0
airport_30D_min_dep_delay    0
airport_30D_max_dep_delay    0
airport_30D_min_arr_delay    0
airport_30D_max_arr_delay    0
carrier_30D_avg_dep_delay    0
carrier_30D_avg_arr_delay    0
carrier_30D_min_dep_delay    0
carrier_30D_max_dep_delay    0
carrier_30D_min_arr_delay    0
carrier_30D_max_arr_delay    0
airport_30D_avg_taxi_out     0
airport_30D_avg_taxi_in      0
weather_code_org             0
weather_code_dest            0
dtype: int64

In [15]:
import datetime
X['year'] = pd.DatetimeIndex(X['fl_date']).year
X['month'] = pd.DatetimeIndex(X['fl_date']).month
X['day'] = pd.DatetimeIndex(X['fl_date']).day
X=X.drop('fl_date',axis=1)


In [16]:
y=X_delays[['nas_delay','late_aircraft_delay','carrier_delay','weather_delay','security_delay']]
y

,nas_delay,late_aircraft_delay,carrier_delay,weather_delay,security_delay
0,NaN,NaN,NaN,NaN,NaN
1,8.0,117.0,0.0,0.0,0.0
2,34.0,0.0,97.0,0.0,0.0
3,29.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
96525,NaN,NaN,NaN,NaN,NaN
96526,NaN,NaN,NaN,NaN,NaN
96527,NaN,NaN,NaN,NaN,NaN
96528,NaN,NaN,NaN,NaN,NaN


In [17]:
delay_type=[]
for i in y.index:
    if y.loc[i].isnull().sum()==5:
        delay_type.append('n/a')
    else:
        ini=0
        for delay in y.loc[i].index:
            if y.loc[i].loc[delay]>ini:
                ini=y.loc[i].loc[delay]
                d=delay
            else:
                pass
        delay_type.append(d)
    

In [18]:
y=pd.DataFrame(delay_type,columns=['delay_type']).reset_index(drop=True)
y=y.replace({
    'n/a':0,
    'late_aircraft_delay':1,
    'nas_delay':2,
'carrier_delay':3,
'weather_delay':4,
'security_delay':5
})

In [19]:
y=y['delay_type']

In [21]:
X.columns

Index(['origin_airport_id', 'dep_hr', 'arr_hr', 'carrier', 'air_time',
       'distance', 'dest_airport_id', 'distance', 'airport_30D_avg_dep_delay',
       'airport_30D_avg_arr_delay', 'airport_30D_min_dep_delay',
       'airport_30D_max_dep_delay', 'airport_30D_min_arr_delay',
       'airport_30D_max_arr_delay', 'carrier_30D_avg_dep_delay',
       'carrier_30D_avg_arr_delay', 'carrier_30D_min_dep_delay',
       'carrier_30D_max_dep_delay', 'carrier_30D_min_arr_delay',
       'carrier_30D_max_arr_delay', 'airport_30D_avg_taxi_out',
       'airport_30D_avg_taxi_in', 'weather_code_org', 'weather_code_dest',
       'year', 'month', 'day'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split
from sklearn import tree 

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
results_dict = {'Classifier':[],
                'Train Accuracy':[], 
                'Test Accuracy':[]
               }

models = {
    'knn'           : KNeighborsClassifier(),
    'decision tree' : DecisionTreeClassifier(),
    'random forest' : RandomForestClassifier(n_estimators=10),
    'SVM'           : SVC(gamma='scale'),
    'logistic regression': LogisticRegression(),
    #'Naive Bayes' : MultinomialNB()
}                                                    

In [22]:
# Looping through models
for model_name, model in models.items():
    print("Fitting %s..." % model_name)
    model.fit(X_train, y_train);
    train_accuracy = model.score(X_train, y_train)*100
    test_accuracy = model.score(X_test, y_test)*100
    results_dict['Classifier'].append(model_name)
    results_dict['Train Accuracy'].append(train_accuracy)
    results_dict['Test Accuracy'].append(test_accuracy)  
    
results_df = pd.DataFrame(results_dict)

Fitting knn...
Fitting decision tree...
Fitting random forest...
Fitting SVM...
Fitting logistic regression...


In [23]:
results_df

,Classifier,Train Accuracy,Test Accuracy
0,knn,77.366030,73.206948
1,decision tree,99.992600,63.686591
2,random forest,97.729795,74.301599
3,SVM,74.619289,74.329224
4,logistic regression,74.619289,74.329224


In [24]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [25]:
# KNN
knn = KNeighborsClassifier()
knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(X_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_
knears_neighbors


KNeighborsClassifier(n_neighbors=4)

In [26]:
knn=knears_neighbors
knn.fit(X_train,y_train)
y_knn_pred=knn.predict(X_test)

In [27]:
from sklearn.metrics import mean_squared_error
from collections import Counter

In [28]:
Counter(y_knn_pred)

Counter({0: 26871, 1: 950, 3: 349, 2: 727, 4: 62})

In [29]:
model.score(X_test, y_knn_pred)

0.9278980627784108

In [30]:
#logistic
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)

log_reg = grid_log_reg.best_estimator_

In [31]:
log_reg

LogisticRegression(C=0.001)

In [32]:
lgr_opt=log_reg
log_reg.fit(X_train,y_train)
y_log_pred=log_reg.predict(X_test)
model.score(X_test, y_log_pred)

1.0

In [33]:
Counter(y_log_pred)

Counter({0: 28959})